In [18]:
import pandas as pd

In [19]:
import pandas as pd
import os
from os.path import isfile, join
import time
import re
import numpy as np
import pandas as pd
import re
import pickle
import pandas as pd
import itertools
from collections import Counter
import numpy as np
from nltk import word_tokenize
from nltk.corpus import stopwords
from gensim.models import word2vec
from sklearn.linear_model import LogisticRegression
import os
import string

In [20]:
train_total = pd.read_csv('train_processed.csv')
test_total = pd.read_csv('test_processed.csv')

In [22]:
import nltk
nltk.download('punkt') # downloads a model
nltk.download('stopwords') # <--- this is new
from nltk.corpus import stopwords
stop = set(stopwords.words('english'))
from nltk.stem import PorterStemmer 

ps = PorterStemmer() 

# return a list of tokens
def pre_processing_by_nltk(doc, stemming = True, need_sent = False):
    # step 1: get sentences 
    #sentences = re.sub(r'[^\w\s]', ' ', doc)
    #sentences = sent_tokenize(sentences)
    # step 2: get tokens
    #tokens = []
    #for sent in sentences:
    words = word_tokenize(doc)
        # step 3 (optional): stemming
    if stemming:
        words = [ps.stem(word) for word in words]
#     if need_sent:
#         tokens.append(words)
#     else:
#         tokens += words
    return [w.lower() for w in words]

[nltk_data] Downloading package punkt to /home/jovyan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [23]:
tagged_data = [word_tokenize(_d) for i, _d in enumerate(train_total["text"])]

In [24]:
from collections import defaultdict
freq = defaultdict(int)
for doc in tagged_data:
    for token in doc:
        freq[token] += 1

In [25]:
freq_copy = freq.copy()
for key, value in freq_copy.items():
    if value < 30 or value > 25000 :
        del freq[key]

In [28]:
from math import log
IDF, vocab = dict(), dict()
for token in freq:
    vocab[token] = len(vocab) #create a fix index of all words
    IDF[token] = log(1 + len(train_total) / freq[token]) #

In [29]:
def vocabFreq(doc,freqdic):
    tokens = doc.split(' ')
    #tokens = pre_processing_by_nltk(doc)
    x= [0]*7911 #vector size equal to
    cor = list(freqdic.keys())
    for i in range(len(freqdic.keys())):
        if cor[i] in tokens:
            x[i] = freqdic[cor[i]]
    return x

In [30]:
train_total = train_total.sample(frac = 1)

In [32]:
X_train_2 = []
for doc in train_total['text']: #create a feature vector 
    X_train_2.append(vocabFreq(doc, freq))
    if len(X_train_2) == 100:
        print(100)
    if len(X_train_2) == 1000:
        print(1000)
    if len(X_train_2) == 10000:
        print(1000)

100
1000
1000


In [24]:
from sklearn.linear_model import LogisticRegression
M2 = LogisticRegression(random_state=0,max_iter=10000, C = 1,n_jobs = -1,verbose = 1).fit(X_train_2,train_total['label'])

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:  2.5min finished


In [33]:
X_test_2 = []
for doc in test_total['text']: #create a feature vector 
    X_test_2.append(vocabFreq(doc, freq))
    if len(X_test_2) == 100:
        print(100)
    if len(X_test_2) == 1000:
        print(1000)
    if len(X_test_2) == 10000:
        print(1000)

100
1000
1000


In [26]:
predict_y2 = M2.predict(X_test_2)

In [29]:
import sklearn
macro_f1_2 = sklearn.metrics.f1_score(test_total['label'], predict_y2,average='macro')
micro_f1_2 = sklearn.metrics.f1_score(test_total['label'], predict_y2,average='micro')
print('macro F1 is ',macro_f1_2, 'micr F1 is ',micro_f1_2)


macro F1 is  0.822838047250092 micr F1 is  0.82284


In [30]:
auc2 = sklearn.metrics.roc_auc_score(test_total['label'], M2.predict_proba(X_test_2)[:, 1])
auc2

0.9002813952

In [33]:
df_tf_result = pd.DataFrame(
    {'label': test_total['label'],
     'predict': M2.predict_proba(X_test_2)[:, 1]})

In [35]:
df_tf_result.to_csv('df_tf_result.csv')

In [34]:
import numpy as np
from sklearn.feature_extraction.text import TfidfTransformer
np.set_printoptions(precision=2)
tfidf = TfidfTransformer(use_idf=True, norm='l2', smooth_idf=True)

tfidf.fit(X_train_2)
Xtrain_TFIDF = tfidf.transform(X_train_2)

In [16]:
def tfidf_feature_extractor(doc, vocab, IDF):
    tokens = doc.split(' ')
    x = [0]*7911
    for token in set(tokens):
        if token in freq.keys():
            tfidf = freq[token] * IDF[token]
            token_id = vocab[token]
            x[token_id] = tfidf # this will be a dense matrix
    return x

In [50]:
X_train_3 = []
X_test_3 = []

for doc in train_total['text']:
    X_train_3.append(tfidf_feature_extractor(doc, vocab, IDF))

for doc in test_total['text']:
    X_test_3.append(tfidf_feature_extractor(doc, vocab, IDF))  

In [72]:
from sklearn.linear_model import LogisticRegression
M3 = LogisticRegression(random_state=0,max_iter=1e7, C = 5,verbose = 1e-3).fit(X_train_3,train_total['label'])


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  8.6min finished


In [76]:
predict_y3 = M3.predict(Xtest_TFIDF)

In [78]:
macro_f1_3 = sklearn.metrics.f1_score(test_total['label'], predict_y3,average='macro')
micro_f1_3 = sklearn.metrics.f1_score(test_total['label'], predict_y3,average='micro')
print('macro F1 is ',macro_f1_3, 'micr F1 is ',micro_f1_3)

macro F1 is  0.7656016772483888 micr F1 is  0.77252


In [79]:
auc2 = sklearn.metrics.roc_auc_score(test_total['label'], M3.predict_proba(Xtest_TFIDF)[:, 1])
auc2

0.8957381056

In [65]:
df_tfidf_result = pd.DataFrame(
    {'label': test_total['label'],
     'predict': M3.predict_proba(Xtest_TFIDF)[:, 1]})

In [66]:
df_tfidf_result.to_csv('df_tfidf_result.csv',index = False)